# Neighborhood Council Service Regions

I looked at empowerla.org.  I decided to synchronize my data with what they have.

Steps in this nb:

  1.  Background from empowerla
  2.  Read the NC data
  3.  Display the NC's
  4.  Display the service regions
  5.  Fix the regions
  6.  Save the clean data

In [2]:
%run start.py

import nc

2021-11-29 17:28:02 Configured OSMnx 1.1.1
2021-11-29 17:28:02 HTTP response caching is on


# 1 - Background

In [3]:
IFrame("https://lahub.maps.arcgis.com/apps/instant/lookup/index.html?appid=e4ab1b3373914c7d99ade15d6f3b9f05", width=1400, height=800)

In [4]:
IFrame("https://empowerla.org/councils-by-service-region/", width=1400, height=800)

 Julien's debrief charts give us an idea of NC's particpating so far.
 
 <br><br>
 <div align="center">
    <img src="../img/debrief.png" width="75%" height="75%">
 </div>

# 2 - Neighborhood Council Data

There seem to be multiple datasets with the NC polygons.  After some poking I found the certified shape file.  It includes a mapping of service regions to nc's.

In [5]:
neighborhoods_gdf = gpd.read_file('../data/neighborhoods/Neighborhood_Councils_(Certified).zip')

# 3 - Map Display

Let's look at the NC's without adding the servcie regions.

**Note** - Not sure why I don't add the region_id color_code at this point?

In [6]:
nc_layer = GeoData(geo_dataframe = neighborhoods_gdf,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.8, 'weight':1.9, 'dashArray':'5', 'fillOpacity':0.2},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Neighborhood Council')

#map_display.add_layer(geo_data)
#map_display += nc_layer


In [7]:
neighborhoods_gdf['region_id'] = neighborhoods_gdf.apply(lambda row: row['SERVICE_RE'].split(' ')[1], axis=1)

In [8]:
neighborhoods_gdf['color_code'] = neighborhoods_gdf.apply(lambda row: nc.empowerla_groups[int(row['region_id'])].color_code, axis=1)

In [9]:
neighborhoods_gdf.query(f"NC_ID == 111")

,OBJECTID,NAME,WADDRESS,DWEBSITE,DEMAIL,DPHONE,NC_ID,CERTIFIED,TOOLTIP,NLA_URL,SERVICE_RE,geometry,region_id,color_code
53,54,NORTH HILLS WEST NC,http://www.nhwnc.net/,http://empowerla.org/NHWNC,NHWNC@EmpowerLA.org,213-978-1551,111,2003-02-25,NORTH HILLS WEST NC,navigatela/reports/nc_reports.cfm?id=111,REGION 1 - NORTH EAST VALLEY,"POLYGON ((-118.47235 34.25012, -118.47231 34.2...",1,#00BFFF


Here's the map of the Neighborhood Councils

In [10]:
imagery = basemap_to_tiles(basemaps.Esri.WorldImagery)
imagery.base = True
osm = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
osm.base = True


map_display = Map(center=(34.05, -118.25), zoom=12,
                  layers=[imagery, osm],
                  layout=Layout(height="700px"),
                  scroll_wheel_zoom=True)

map_display.add_control(LayersControl())
map_display += nc_layer
map_display

Map(center=[34.05, -118.25], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

# 4 - Display Service Regions

**Note** This is the cell to execute after the "fix".

In [11]:
a_geojson = json.loads(neighborhoods_gdf.to_json())

def region_color(feature):
    return {
        'color': 'black',
        'fillColor': feature['properties']['color_code']
    }

geo_json = GeoJSON(
    data=a_geojson,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.6, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    },
    style_callback=region_color,
    name='Regions'
)

map_display.add_layer(geo_json)

html = HTML('''Hover over a district''')
html.layout.margin = '0px 20px 20px 20 px'
control = WidgetControl(widget=html, position='bottomright')

def update_html(feature, **kwargs):
    html.value = '''<h3><b>NC: {}</b></h3>
                    <h4>NC ID: {}</h4>
                    <h4>region id: {}</h4>'''.format(feature['properties']['NAME'], 
                                                     feature['properties']['NC_ID'],
                                                     feature['properties']['region_id'])
    
map_display.add_control(control)  # does += work for this?

geo_json.on_hover(update_html)

Go back and look at the map.  Compare it with the map from empowerla in the background section and you should see the problem.

# 5 - Fix and Display

After some exploration we can see the problem.  This section resets the map then cleans up the gdf using the pythonic idiom to swap values.

In [ ]:
map_display.remove_layer(geo_json)
map_display.remove_control(control)

In [ ]:
neighborhoods_gdf.loc[52, 'SERVICE_RE'], neighborhoods_gdf.loc[53, 'SERVICE_RE'] = \
neighborhoods_gdf.loc[53, 'SERVICE_RE'], neighborhoods_gdf.loc[52, 'SERVICE_RE']

neighborhoods_gdf['region_id'] = neighborhoods_gdf.apply(lambda row: row['SERVICE_RE'].split(' ')[1], axis=1)

neighborhoods_gdf['color_code'] = neighborhoods_gdf.apply(lambda row: nc.empowerla_groups[int(row['region_id'])].color_code, axis=1)

Go back and execute the section 4 cell.  Once you do that go look at the map to display the updated service regions.

Looks better!

# 6 - Save Clean File

In [ ]:
neighborhoods_gdf.to_file('../data/neighborhoods/Neighborhood_Councils_(Certified)_cleaned.zip')

**Note** - now we have the cleaned NC shape file (as a zip).